In [0]:
%sql CREATE SCHEMA IF NOT EXISTS  silver 

In [0]:
df = spark.table("workspace.bronze.solar_production")

In [0]:
from pyspark.sql.functions import posexplode, to_date, regexp_extract, col

df_exploded = df.select(
    posexplode("energy").alias("hour", "kwh"),
    to_date(regexp_extract("file_name", '(\\d{4}-\\d{2}-\\d{2})', 1),'yyyy-MM-dd').alias("date")
)
display(df_exploded)

In [0]:
from pyspark.sql.functions import year, month, dayofmonth, make_timestamp, col, lit

df_with_datehour = df_exploded.withColumn(
    "datehour",
    make_timestamp(
        year(col("date")),
        month(col("date")),
        dayofmonth(col("date")),
        col("hour"),
        lit(0),
        lit(0)
    )
)

In [0]:
from pyspark.sql.functions import col
df_hourly = df_with_datehour.select(
    col("datehour"),
    col("kwh")
)
df_hourly.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver.fct_hourly_kwh")
display(df_hourly)